# Ollama Function Calling

There are 2 mechanism for function calling with Ollama. There is the _traditional_ Ollama method. It can also support the _OpenAI_ function calling API. I am trying both in this instance.

First, let's define a dummy function that can get the weather.

In [1]:
def get_weather(location: str)->str:    
    '''Get current temperature for a given location.'''
    print(f"Getting weather for: {location}")
    return "Temperature is 10 degrees"

Next, let's create a dictionary of available functions.

In [12]:
available_functions = {
    "get_weather": get_weather
}

## Ollama Method
Here is function calling using the traditional Ollama method. Note that the initial call does not actually _invoke_ the function. This has to happen outside of the `ollama.chat` call. So you need to parse the response and then make the function call.

In [14]:
import ollama
response = ollama.chat(
    model='llama3.2:latest',
    messages=[{'role': 'user', 'content':
        'What is the weather in Toronto?'}],

		# provide a weather checking tool to the model
    tools=[{
      'type': 'function',
      'function': {
        'name': 'get_weather',
        'description': 'Get the current weather for a location',
        'parameters': {
          'type': 'object',
          'properties': {
            'location': {
              'type': 'string',
              'description': 'The name of the city, country. For example, London, United Kingdom.',
            },
          },
          'required': ['location'],
        },
      },
    },
  ],
)

print(response['message']['tool_calls'])

[{'function': {'name': 'get_weather', 'arguments': {'location': 'Toronto'}}}]


In [15]:
for tool in response["message"]["tool_calls"]:
    function_to_call = available_functions[tool['function']['name']]
    function_args = tool['function']['arguments']
    function_response = function_to_call(**function_args)
    print(function_response)

Getting weather for: Toronto
Temperature is 10 degrees


## OpenAI Method
Now let's switch to the OpenAI API. If you want to develop and test your code on local host and then deploy to OpenAI, this is a much better mechanim. Again, the initial call only returns an object for tool calling. You would have to parse the result and actually call the function.

In [16]:
import json
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. London, United Kingdom"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="llama3.2:latest",
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print(completion.choices[0].message.tool_calls)
function_to_call = available_functions[completion.choices[0].message.tool_calls[0].function.name]
function_args = json.loads(completion.choices[0].message.tool_calls[0].function.arguments)
function_response = function_to_call(**function_args)
print(function_response)

[ChatCompletionMessageToolCall(id='call_g8of7tm5', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function', index=0)]
Getting weather for: Paris, France
Temperature is 10 degrees
